In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os

In [ ]:
def load_generation_data(include_files=None, date_filter=None):
    data_dir = "/Users/jangseohyun/Documents/workspace/symply/DER/data/generation"
    all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

    if include_files is not None:
        for file in include_files:
            if file not in all_files:
                raise ValueError(f"파일을 찾을 수 없습니다: {file}")
        files_to_load = [f for f in all_files if f in include_files]
    else:
        files_to_load = all_files

    I = len(files_to_load)  
    T = 24 
    generation_data = np.zeros((I, T)) 

    loaded_files = []
    
    for idx, file in enumerate(files_to_load):
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()

        date_col = "Date"
        hour_col = "Hour (Eastern Time, Daylight-Adjusted)"
        gen_col = "Electricity Generated"

        if any(col not in df.columns for col in [date_col, hour_col, gen_col]):
            print(f"{file}: 필요한 컬럼 없음. 스킵됨.")
            continue

        if date_filter:
            df = df[df[date_col] == date_filter]
            if df.empty:
                print(f"{file}: {date_filter} 데이터 없음. 스킵됨.")
                continue 

        df = df[df[hour_col].astype(str).str.match(r'^\d+$')]
        df["Time"] = df[hour_col].astype(int)
        df = df[df["Time"].between(0, 23)]

        for t in range(T):
            if t in df["Time"].values:
                generation_data[idx, t] = df[df["Time"] == t][gen_col].values[0]

        loaded_files.append(file)

    print(f"✅ 총 {I}개 파일을 불러왔습니다: {', '.join(loaded_files)}")

    return generation_data, I, T

def generate_randomized_generation(I, T, S, data, randomness_level):
    np.random.seed(1)

    noise_ranges = {
        "low": (0.8, 1.2),
        "medium": (0.5, 1.5),
        "high": (0.2, 1.8),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(I, T, S))

    generation_r = np.expand_dims(data, axis=-1) * noise_factors
    
    print(f"📊 데이터 Shape: I={I}, T={T}, S={S}")
    return generation_r

def plot_generation_data(generation_data, I):
    hours = np.arange(24)
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, generation_data[i], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title("Hourly Electricity Generation for All Generators")
    plt.xticks(hours)  # 0~23 시간 설정
    plt.legend(loc="upper left", fontsize='small')

    plt.show()

def plot_randomized_generation(R, I, T, S):
    hours = np.arange(T)
    
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, R[i, :, S], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Randomized Hourly Generation for Scenario {S}")
    plt.xticks(hours) 
    plt.legend(loc="upper left") 

    plt.show()
       
def plot_scenarios_for_generator(R, i):

    T = R.shape[1]
    S = R.shape[2] 
    hours = np.arange(T) 

    plt.figure(figsize=(15, 9))

    for s in range(S):
        plt.plot(hours, R[i, :, s], linestyle='-', alpha=0.6, label=f'Scenario {s+1}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Hourly Electricity Generation for Generator {i} Across All Scenarios")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize='small', ncol=2)
    plt.show()

def generate_rt_scenarios(rt_da, S, randomness_level):

    rt_da["Time Stamp"] = pd.to_datetime(rt_da["Time Stamp"])
    nyc_rt = rt_da[rt_da["Name"] == "N.Y.C."].copy() 

    # Extract the start of the day and filter only the first 24 hours
    start_of_day = nyc_rt["Time Stamp"].min().floor("D")
    end_of_day = start_of_day + pd.Timedelta(hours=23)
    nyc_rt = nyc_rt[(nyc_rt["Time Stamp"] >= start_of_day) & (nyc_rt["Time Stamp"] <= end_of_day)]

    nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")
    hourly_avg = nyc_rt.groupby("Hour")["LBMP ($/MWHr)"].mean().reset_index()
    price_hourly = hourly_avg["LBMP ($/MWHr)"].to_numpy()
    T = len(price_hourly)

    np.random.seed(11)
    noise_ranges = {
        "low": (0.95, 1.05),
        "medium": (0.85, 1.15),
        "high": (0.7, 1.3),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Choose from 'low', 'medium', 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(T, S))

    P_RT = np.expand_dims(price_hourly, axis=-1) * noise_factors

    return P_RT

def plot_rt_scenarios(P_RT):
    T, S = P_RT.shape
    hours = np.arange(T)

    plt.figure(figsize=(15, 8))

    for s in range(S):
        plt.plot(hours, P_RT[:, s], linestyle='-', alpha=0.6, label=f"Scenario {s+1}")

    plt.xlabel("Hour")
    plt.ylabel("Price ($/MWHr)")
    plt.title("Real-Time Price Scenarios (Hourly Averaged)")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize="small", ncol=2)

    plt.show()
    
only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
ny_da = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718da.csv")
ny_rt = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718rt.csv")
ny_da["Time Stamp"] = pd.to_datetime(ny_da["Time Stamp"])
ny_da["Hour"] = ny_da["Time Stamp"].dt.hour
nyc_data = ny_da[ny_da["Name"] == "N.Y.C."]
P_DA = nyc_data["LBMP ($/MWHr)"].to_numpy() * 1.3
P_PN = P_DA * 1.5

# plot_generation_data(generation_data, 10)
# plot_randomized_generation(R, I, T, 7)
# plot_scenarios_for_generator(R, 1)
# plot_rt_scenarios(P_RT)


✅ 총 10개 파일을 불러왔습니다: 1201.csv, 137.csv, 281.csv, 397.csv, 401.csv, 430.csv, 514.csv, 524.csv, 775.csv, 89.csv
📊 데이터 Shape: I=10, T=24, S=100


/var/folders/2_/9m9qjphn0xs45xzpvf1hl0j00000gn/T/ipykernel_23304/577363375.py:131: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


### 모델 DER only

In [ ]:
include_files = ['1201.csv', '137.csv', '281.csv', '397.csv', '401.csv', '430.csv', '514.csv', '524.csv', '775.csv', '89.csv']
# include_files = ['1201.csv', '137.csv', '401.csv', '524.csv', '89.csv']
generation_data, I, T = load_generation_data(include_files, "2022-07-18")

S = 50
randomness_level = "high"
R = generate_randomized_generation(I, T, S, generation_data, randomness_level)
P_RT = generate_rt_scenarios(ny_rt, S, randomness_level)

In [176]:
only = gp.Model("only")
# only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")

M = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")

only.update()

obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * y_plus[i, t, s] - P_PN[t] * y_minus[i, t, s])
    for i in range(I)
    for t in range(T)
    for s in range(S)
)

only.setObjective(obj, GRB.MAXIMIZE)

In [177]:
for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(y_plus[i, t, s] <= R[i, t, s])

# for i in range(I):
#     for t in range(T):
#         for s in range(S):
#             only.addConstr(y_plus[i, t, s] <= M * z[i, t, s])
#             only.addConstr(y_minus[i, t, s] <= M * (1 - z[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48000 rows, 72240 columns and 96000 nonzeros
Model fingerprint: 0xae2c85a1
Variable types: 48240 continuous, 24000 integer (24000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 8e+02]
Found heuristic solution: objective 1725742.1991
Presolve removed 47913 rows and 72153 columns
Presolve time: 0.01s
Presolved: 87 rows, 87 columns, 87 nonzeros
Found heuristic solution: objective 2069959.2908
Variable types: 87 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.03 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 2.06996e+06 1.72574e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.069959290

### 결과 분석

#### 수익 분석

In [178]:
# Day-ahead 수익 비교
total_da_profit_obj = 0
for i in range(I):
    for t in range(T):
        total_da_profit_obj += P_DA[t] * x[i,t].x

# Real-time 수익 비교
total_rt_profit_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            rt_profit_obj = P_RT[t, s] * y_plus[i, t, s].x
            total_rt_profit_obj += 1/S * rt_profit_obj

# 패널티 비용 비교
total_penalty_cost_obj = 0
for i in range(I):
    for t in range(T):
        for s in range(S):
            penalty_cost_obj = P_PN[t] * y_minus[i, t, s].x
            total_penalty_cost_obj += 1/S * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익 (_obj): {total_system_profit_obj:.2f}")

DA: 413631.43
RT: 1711200.56
Penalty: 54872.70
목적 함수 기반 총 이익 (_obj): 2069959.29


#### 하루 개인별 커밋량 합 분석

In [179]:
sum_x = 0
for t in range(T):
    time_sum = sum(x[i,t].x for i in range(I))
    print(f"시간 {t}: {time_sum}")
    sum_x += time_sum
print(f"총 하루 commitment: {sum_x:.2f}")


시간 0: 0.0
시간 1: 0.0
시간 2: 0.10947853916716899
시간 3: 0.0
시간 4: 0.28827460563547536
시간 5: 0.0
시간 6: 4.834572326332672
시간 7: 12.563528328023315
시간 8: 40.783872445356415
시간 9: 171.64145777030168
시간 10: 324.519962460199
시간 11: 0.0
시간 12: 0.0
시간 13: 0.0
시간 14: 1213.5716817124637
시간 15: 0.0
시간 16: 581.5891792193593
시간 17: 0.0
시간 18: 0.0
시간 19: 275.61731973125393
시간 20: 91.84375921479497
시간 21: 19.91667422452679
시간 22: 0.0
시간 23: 0.0
총 하루 commitment: 2737.28


#### 개인 수익 분석

In [180]:
# 모든 der의 profit 합계 계산
total_der_profit = 0
der_profit = {}
for i in range(I):
    # 각 der i의 profit 합계 계산
    der_profit[i] = sum(P_DA[t] * x[i,t].x + sum(1/S * (P_RT[t,s] * y_plus[i,t,s].x - P_PN[t] * y_minus[i,t,s].x) for s in range(S)) for t in range(T))
    total_der_profit += der_profit[i]

print("\nder_profit:")
for i in range(I):
    print(f"[{i}] {der_profit[i]:.2f}")

print(f"\n모든 der의 profit 합계: {total_der_profit:.2f}")

der_hourly_profit = np.zeros((I, T, 4))

for i in range(I):
    for t in range(T):
        da_profit = P_DA[t] * x[i, t].x
        rt_profit = sum(1 / S * (P_RT[t, s] * y_plus[i, t, s].x) for s in range(S))
        pen_cost = sum(1 / S * (P_PN[t] * y_minus[i, t, s].x) for s in range(S))
        hourly_total = da_profit + rt_profit - pen_cost

        der_hourly_profit[i, t, 0] = da_profit  # DA profit
        der_hourly_profit[i, t, 1] = rt_profit  # RT profit
        der_hourly_profit[i, t, 2] = pen_cost   # Penalty cost
        der_hourly_profit[i, t, 3] = hourly_total



der_profit:
[0] 201347.07
[1] 327157.04
[2] 183947.77
[3] 50055.56
[4] 389352.32
[5] 91237.92
[6] 85988.17
[7] 455734.95
[8] 126851.08
[9] 158287.41

모든 der의 profit 합계: 2069959.29


### 결과 저장

In [181]:
# R_sum = {(i, t): sum(R[i, t, s] for s in range(S)) / S for i in range(I) for t in range(T)}
# R_df = pd.DataFrame({i: {t: R_sum[i, t] for t in range(T)} for i in range(I)})
# R_df.to_csv('result_R.csv', index=False)

# pd.DataFrame(list(P_DA.items()), columns=['time', 'price']).to_csv('result_P_DA.csv', index=False)

# total_x_only = {t: sum(x[i,t].x for i in range(I)) for t in range(T)}
# total_x_only_df = pd.DataFrame({'total_x_only': total_x_only})
# total_x_only_df.to_csv('result/result_base_totalX.csv', index=False)

only_obj = pd.DataFrame({'only_obj': [only.objVal]})
only_obj.to_csv('result/result_only_obj.csv', index=False)

only_profit_value = pd.DataFrame({'only_profit_value': der_profit})
only_profit_value.to_csv('result/result_only_profit.csv', index=False)

data_list = []

for i in range(I):
    for t in range(T):
        data_list.append([i, t, 
                          der_hourly_profit[i, t, 0],  # DA profit
                          der_hourly_profit[i, t, 1],  # RT profit
                          der_hourly_profit[i, t, 2],  # Penalty cost
                          der_hourly_profit[i, t, 3]]) # Hourly total profit

# Pandas DataFrame 생성
df = pd.DataFrame(data_list, columns=['DER', 'Hour', 'hourly_da', 'hourly_rt', 'hourly_pen', 'hourly_total'])

# CSV 파일로 저장
df.to_csv('result/result_only_hourly_profit.csv', index=False)
